In [3]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr 12 16:43:24 2020

@author: csyvi
"""

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
import warnings
warnings.filterwarnings("ignore")

#read in the dataset
df = pd.read_csv('finalOutput_scored.csv')
#take a look at the data
df.head()

df.shape

X = df.drop(columns = ['user id','tweets content', 'created on', 'clean_tweet', 'sentiment_classification'])
X = X.loc[0:1999, :]
y = df['sentiment_classification']
y = y.to_frame()
y = y.loc[0:1999, :]

x_all = df.drop(columns = ['user id','tweets content', 'created on', 'clean_tweet', 'sentiment_classification'])

y.isnull().sum()    

#split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

#create new a knn model
knn = KNeighborsClassifier()
#create a dictionary of all values we want to test for n_neighbors
params_knn = {'n_neighbors': np.arange(1, 25)}
#use gridsearch to test all values for n_neighbors
knn_gs = GridSearchCV(knn, params_knn, cv=5)
#fit model to training data
knn_gs.fit(X_train, y_train)

#save best model
knn_best = knn_gs.best_estimator_
#check best n_neigbors value
print(knn_gs.best_params_)

#create a new random forest classifier
rf = RandomForestClassifier()
#create a dictionary of all values we want to test for n_estimators
params_rf = {'n_estimators': [50, 100, 200]}
#use gridsearch to test all values for n_estimators
rf_gs = GridSearchCV(rf, params_rf, cv=5)
#fit model to training data
rf_gs.fit(X_train, y_train)

#save best model
rf_best = rf_gs.best_estimator_
#check best n_estimators value
print(rf_gs.best_params_)


#create a new logistic regression model
log_reg = LogisticRegression()
#fit the model to the training data
log_reg.fit(X_train, y_train)


#test the three models with the test data and print their accuracy scores
print('knn: {}'.format(knn_best.score(X_test, y_test)))
print('rf: {}'.format(rf_best.score(X_test, y_test)))
print('log_reg: {}'.format(log_reg.score(X_test, y_test)))

#create a dictionary of our models
estimators=[('knn', knn_best), ('rf', rf_best), ('log_reg', log_reg)]

#create our voting classifier, inputting our models
ensemble = VotingClassifier(estimators, voting='hard')

#fit model to training data
ensemble.fit(X_train, y_train)
#test our model on the test data
ensemble.score(X_test, y_test)

prediction = ensemble.predict(x_all)

dfPredict = pd.DataFrame(prediction)

df['ensemble_predict'] = dfPredict

df.head()

df.shape

df.to_csv('finalOutput_scored_ensemblePredicted.csv')

{'n_neighbors': 3}
{'n_estimators': 50}
knn: 0.915
rf: 0.9116666666666666
log_reg: 0.915
